Download audio using pytube

In [ ]:
from pytube import YouTube

# Enter the URL of the YouTube video
video_url = "https://www.youtube.com/watch?v=lks-x8ZM554"

# Create a YouTube object
yt = YouTube(video_url)

# Get the audio stream
audio_stream = yt.streams.filter(only_audio=True).first()

# Download the audio stream
audio_directory = "jeff dean leaders connect bangalore"
audio_stream.download(output_path=f"./{audio_directory}", filename="audio.mp4")

Generate transcript using whisperX

In [ ]:
# !pip install -U openai-whisper
# sudo apt update && sudo apt install ffmpeg

In [2]:
# import whisper

# model = whisper.load_model("base")

# # Specify the path to the downloaded audio file
# audio_file_path = f"./{audio_directory}/audio.mp4"

# # Generate the transcript
# transcript = model.transcribe(audio_file_path)
# # transcript = model.transcribe("audio_sample.mp4")

# # Print the transcript
# print(transcript)


In [7]:
import whisperx
import gc 

device = "cpu" 
audio_file = "audio_fireside_chat.mp4"
batch_size = 4 # reduce if low on GPU mem
compute_type = "int8" # change to "int8" if low on GPU mem (may reduce accuracy)

# 1. Transcribe with original whisper (batched)
model = whisperx.load_model("base", device, compute_type=compute_type)

# save model to local path (optional)
# model_dir = "/path/"
# model = whisperx.load_model("large-v2", device, compute_type=compute_type, download_root=model_dir)

audio = whisperx.load_audio(audio_file)
result = model.transcribe(audio, batch_size=batch_size)
print(result["segments"]) # before alignment

# delete model if low on GPU resources
# import gc; gc.collect(); torch.cuda.empty_cache(); del model

# 2. Align whisper output
model_a, metadata = whisperx.load_align_model(language_code=result["language"], device=device)
result = whisperx.align(result["segments"], model_a, metadata, audio, device, return_char_alignments=False)

print(result["segments"]) # after alignment

# delete model if low on GPU resources
# import gc; gc.collect(); torch.cuda.empty_cache(); del model_a

Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.1.4. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../.cache/torch/whisperx-vad-segmentation.bin`


No language specified, language will be first be detected for each audio file (increases inference time).
Model was trained with pyannote.audio 0.0.1, yours is 3.0.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.0.0. Bad things might happen unless you revert torch to 1.x.
Detected language: en (0.90) in first 30s of audio...
[{'text': " Lower to you. Thank you. Hello, everybody. Welcome. Welcome again, Jeff. Promote. I hope that you will indulge me as I introduce you to the audience with a quick set of rapid fire questions. Jeff, you've been in Bangalore day or two. I hope you're prepared with your answers. Is that okay?", 'start': 0.009, 'end': 25.52}, {'text': " Okay, first one. Coffee or tea? I've had both today, but I'm generally coffee. Coffee or tea? Okay, second one. Masala dosa or avocado toast? Masala dosa. Good one. Is there something called avocado toast? You're a morning person or an evening person. N

In [ ]:
# # 3. Assign speaker labels
# diarize_model = whisperx.DiarizationPipeline(use_auth_token="", device=device)

# # add min/max number of speakers if known
# diarize_segments = diarize_model(audio, max_speakers=1)
# # diarize_model(audio, min_speakers=min_speakers, max_speakers=max_speakers)

# result = whisperx.assign_word_speakers(diarize_segments, result)
# print(diarize_segments)
# print(result["segments"]) # segments are now assigned speaker IDs

: 

Summarize conversation using Mistal 7B (Langchain and Ollama)

In [ ]:
# from langchain_community.llms import Ollama

# llm = Ollama(model="mistral")
# llm("The first man on the moon was ...")